# Practical Session 6: Hardware deployment with the Turtlebot3 Burger 

## Laboratorio de Robótica 
### Grado en Ingeniería Electrónica, Mecatrónica y Robótica
### Universidad de Sevilla

## Objectives

In this practical session, you will deploy your reactive navigation software on a real robot to test the system in an actual environment. For this purpose, we will use a TurtleBot3 Burger robot—the same model you used in simulation. The objective is for students to transfer their simulated results to the real robot, which involves installing the robot drivers, configuring ROS to work across multiple machines on a local network, and accessing the robot remotely.

In this session, you will learn how to:

+ Set up the TurtleBot3 Burger by installing the required drivers and software packages.
+ Configure the TurtleBot3 and your laptop to use ROS over a WiFi network.
+ Teleoperate the TurtleBot3 from your laptop.
+ Run your ROS package for autonomous navigation on a real TurtleBot3.

## Turtlebot3 features

TurtleBot3 is an open-source (hardware, software, firmware) robotic platform developed for ROS education and research. Its goal is to provide a small, low-cost, and highly flexible robotics development platform without having to sacrifice functionality and quality, while also offering enough expandability to fit a wide variety of complex robotics applications. Specifically, the Burger model has the following components:

+ 360 Laser Distance Sensor LDS-02 for navigation
+ Raspberry Pi Camera
+ OpenCR board with IMU (gyroscope and accelerometer) for motor control
+ Single Board Computer (SBC) for navigation (Raspberry Pi 4)

<figure style="text-align:center">
  <img src="images/turtlebot3_burger_components.png" alt="" width=500>
  <figcaption>Fig. 1: Turtlebot3 Burger's components.</figcaption>
</figure>

In order to make the TurtleBot3 Burger operational, Ubuntu and ROS must be installed on the SBC. After that, you also need to install the drivers for the LDS-02 (laser) and the firmware for the OpenCR board. All these steps are detailed in the Burger [wiki](https://emanual.robotis.com/docs/en/platform/turtlebot3/quick-start).

## Network configuration

In order to operate the robot, you will set up a WiFi network to which the Turtlebot3 SBC and your computer will connect. In this course, each Turtlebot3 has an associated ID number and, depending on the ID of your robot, you will use the following network parameters:
+ WIFI SSID: `ROB_ID`
+ WIFI PASSWORD: `robotica`

You can create this WiFi network using the portable hotspot function of your smartphone. For example, if your Turtlebot3 is number 2, you should create a WiFi named `ROB_2` with password `robotica`. The fact that each robot connects to its own local network ensures that they will not interfere each other. 

By default, each Turtlebot3 will be configured to access its corresponding WiFi network. In case yours is not, or you need to change the WiFi parameters, you can do so by editing the file `/etc/netplan/50-cloud-init.yaml` in your SBC with superuser permissions.

In [ ]:
network:
  version: 2
  renderer: networkd
  wifis:
    wlan0:
      dhcp4: true
      access-points:
        "NAME_OF_YOUR_SSID":
          password: "YOUR_PASSWORD"

Modify the file and apply the changes with `sudo netplan apply`. 

__Note:__ If your Turtlebot3 is not connected to the network, you will have to do this configuration connecting a keyboard and a screen to the SBC.   

### Access the robot computer

Once your Turtlebot3 and laptop are connected to the same WiFi network, you can access the Turtlebot3's SBC via SSH. To do this, you first need to find out its IP address. 
On some smartphones, you can view the IP addresses assigned to the devices connected to the hotspot or WiFi network, these would be the Turtlebot3 and your laptop. If your smartphone does not provide this information, you can use a network-scanning tool on your computer, such as `nmap`, which detects devices on the network and returns their IP addresses. 

Use the `ifconfig` command (Ubuntu/macOS) to find your computer's IP and netmask, or `ipconfig` if you are using Windows. You can download `nmap` from [here](https://nmap.org/), or install it on Ubuntu with `sudo apt install nmap`. Then scan the network using `nmap` to locate the Turtlebot3's IP. You can either open the `nmap` GUI or run the following command in a terminal: `nmap -sn IP/mask`. For example, if your computer has IP 192.168.43.101 and netmask 255.255.255.0, you should scan `nmap -sn 192.168.43.101/24`.

Once you know your Turtlebot3's IP, you can access its computer via SSH specifying the username and IP of the remote machine: `ssh [username]@{IP_OF_TURTLEBOT}`. Depending on the ID of your robot, the SBC will be configured as follows:

+ Hostname: `turtlebotID`
+ Username: `ubuntu`
+ Password: `turtlebot` 

### ROS master configuration

<figure style="text-align:center">
  <img src="images/network_configuration.png" alt="" width=850>
  <figcaption>Fig. 2: Network configuration.</figcaption>
</figure>

When using ROS across multiple machines, each one must know where the ROS master is running. This is configured using the environment variables `ROS_MASTER_URI` and `ROS_HOSTNAME`. The former specifies the IP address where the master is located, while the latter specifies the computer’s own IP address. Configure these variables in the `.bashrc` files of both the Docker container on your laptop and the Turtlebot3. These scripts are executed automatically each time a new terminal is opened.

__Note 1:__ Do not forget to source `.bashrc` to apply the changes.

__Note 2:__ You can also run the Docker container with the following options to avoid editing the `.bashrc` in every session: `--env="ROS_MASTER_URI=http://IP_OF_REMOTE_PC:11311" --env="ROS_HOSTNAME=IP_OF_REMOTE_PC"`.


## Turtlebot3 navigation

Once all the network and ROS configuration is done, you can operate your Turtlebot3 remotely from the Docker container on your laptop. First, run `roscore` on your laptop. Then, open a terminal to access the Turtlebot3 via SSH and launch its drivers with: `roslaunch turtlebot3_bringup turtlebot3_robot.launch`. 

__Exercise 1:__

Test the teleoperation of the Turtlebot3 to navigate the robot manually from your laptop. To do this, run the following command in your ROS container: `roslaunch turtlebot3_teleop turtlebot3_teleop_key.launch`.

__Exercise 2:__

Demonstrate the robot navigating autonomously using your ROS package `autonomous_navigation`. Based on the `challenge_reactive.launch` file you configured for running the system in simulation, prepare a new launch file called `challenge_reactive_real.launch` for the real robot, and upload it to the `launch` folder of your `autonomous_navigation` package. This file will no longer run a Gazebo simulation; instead, it should launch: 

+ 1) The required robot models and coordinate transformations to construct the complete TF tree; 
+ 2) A `path_publisher.py` node that publishes a predefined path:
+ 3) RViz with a pre-configured visualization; 
+ 4) Your nodes for reactive navigation with the Turtlebot3.

Remember that all waypoints must be specified in the robot’s `odom` frame. This frame will act as the TurtleBot3's global reference frame and will correspond to the robot’s initial position and orientation. Consequently, all navigation waypoints will be defined relative to the robot’s starting pose.

## Summary

In this practical session, you should have learned the following:
+ How to set up a Turtlebot3 Burger robot.
+ How to configure a WiFi network to access the Turtlebot3 via SSH.
+ How to configure ROS to work accross multiple machines.
+ How to teleoperate the actual robot and navigate it autonomously using your reactive navigation solution.
